In [1]:
import tensorflow as tf

In [2]:
num_gpus_available = len(tf.config.experimental.list_physical_devices('GPU'))
print("Num GPUs Available: ", num_gpus_available)
assert num_gpus_available > 0

Num GPUs Available:  1


In [3]:
pip install transformers -U

Note: you may need to restart the kernel to use updated packages.


In [4]:
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification

C:\Users\Public\anaconda3\envs\tf-gpu\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [6]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import bz2 # To open zipped files
import re # regular expressions
import os
import gc

import matplotlib.pyplot as plt
import seaborn as sns

from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier

In [8]:
train_file = bz2.BZ2File('extract/train.ft.txt.bz2')
test_file = bz2.BZ2File('extract/test.ft.txt.bz2')

In [9]:
train_file_lines = train_file.readlines()
test_file_lines = test_file.readlines()

In [10]:
del train_file, test_file
gc.collect()

579

In [11]:
train_file_lines = [x.decode('utf-8') for x in train_file_lines]
test_file_lines = [x.decode('utf-8') for x in test_file_lines]

In [12]:
train_labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in train_file_lines]
test_labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in test_file_lines]


In [13]:
print(len(train_labels))
train_labels = train_labels[:100000]
print(len(train_labels))

3600000
100000


In [14]:
print(len(test_labels))
test_labels = test_labels[:100000]
print(len(test_labels))

400000
100000


In [15]:
train_sentences = [x.split(' ', 1)[1][:-1] for x in train_file_lines]
test_sentences = [x.split(' ', 1)[1][:-1] for x in test_file_lines]

In [16]:
print(len(train_sentences))
train_sentences = train_sentences[:100000]
print(len(train_sentences))

3600000
100000


In [17]:
print(len(test_sentences))
test_sentences = test_sentences[:100000]
print(len(test_sentences))

400000
100000


In [18]:
for i in range(len(train_sentences)):
    if 'www.' in train_sentences[i] or 'http:' in train_sentences[i] or 'https:' in train_sentences[i] or '.com' in train_sentences[i]:
        train_sentences[i] = re.sub(r"([^ ]+(?<=\.[a-z]{3}))", "<url>", train_sentences[i])

for i in range(len(test_sentences)):
    if 'www.' in test_sentences[i] or 'http:' in test_sentences[i] or 'https:' in test_sentences[i] or '.com' in test_sentences[i]:
        test_sentences[i] = re.sub(r"([^ ]+(?<=\.[a-z]{3}))", "<url>", test_sentences[i])

In [19]:
list(filter(lambda x: '<url>' in x, train_sentences))[0]

"Bad Deal!!: I ordered this DVD and received a substitute I never received the DVD I ordered from Importcds (the Vendor). I contacted them and did not recieve any feedback. I can't rate a DVD I have never seen. I didn't bother to send it back because it would have cost me more that I orginally paid for it. In the future I will watch for the name of the person and/or persons I am buying from. I thought they were a good company. I understand a simple mistake but, to not get a response at all is not good businees sense. I spend hundreds of dollars a month on <url> building my DVD collection. I guess I will be more careful in the future."

In [20]:
del train_file_lines, test_file_lines
gc.collect()

0

In [21]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [22]:
train_encodings = tokenizer(train_sentences,truncation=True,padding=True)

In [23]:
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings),train_labels))

In [24]:
del train_encodings
gc.collect()

0

In [25]:
print(len(test_labels))
test_labels = test_labels[:30000]
test_sentences = test_sentences[:30000]
print(len(test_sentences))

100000
30000


In [26]:
val_encodings = tokenizer(test_sentences,truncation=True,padding=True)

In [27]:
val_dataset = tf.data.Dataset.from_tensor_slices((dict(val_encodings),test_labels))

In [28]:
del val_encodings
gc.collect()

0

In [29]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=2)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['activation_13', 'vocab_transform', 'vocab_projector', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'pre_classifier', 'dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use i

In [30]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5, epsilon=1e-08)
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])

In [31]:
model.fit(train_dataset.shuffle(100).batch(16),epochs=1,batch_size=16,validation_data=val_dataset.shuffle(100).batch(16))

C:\Users\Public\anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\python\autograph\impl\api.py:376: FutureWarning: The old compute_loss method is deprecated as it conflicts with the Keras compute_loss method added in TF 2.8. If you want the original HF compute_loss, please call hf_compute_loss() instead. From TF versions >= 2.8, or Transformers versions >= 5, calling compute_loss() will get the Keras method instead.
  return py_builtins.overload_of(f)(*args)


ResourceExhaustedError:  OOM when allocating tensor with shape[5984,3072] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node tf_distil_bert_for_sequence_classification/distilbert/transformer/layer_._1/ffn/lin1/Tensordot/MatMul (defined at \Public\anaconda3\envs\tf-gpu\lib\site-packages\transformers\models\distilbert\modeling_tf_distilbert.py:237) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_12897]

Function call stack:
train_function


In [ ]:
test_sentence = "This is a really bad product. I hate it"


In [ ]:
predict_input = tokenizer.encode(test_sentence,truncation=True,padding=True,return_tensors="tf")

In [ ]:
tf_output = model.predict(predict_input)[0]

In [ ]:
tf_prediction = tf.nn.softmax(tf_output, axis=1)
labels = ['Negative','Positive']
label = tf.argmax(tf_prediction, axis=1)
label = label.numpy()
print(labels[label[0]])